In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sys
sys.path.append('../')
import numpy as np
import pandas as pd
from collections import Counter,defaultdict
import csv
import math
import operator

In [3]:
# MODEL_DIR = '/content/drive/MyDrive/OUT-K3/MODELS'
# VOCAB_DIR = '/content/drive/MyDrive/OUT-K3/VOCAB'
# INPUT_DIR='/content/drive/MyDrive/EHR/DATASET'
# DATA_DIR = '/content/drive/MyDrive/EHR/DATA'
# MIMIC_3_DIR = '/content/drive/MyDrive/OUT-K3/MIMIC3'

# MODEL_DIR = '/content/drive/MyDrive/OUT-K3/MODELS'
# VOCAB_DIR = '/content/drive/MyDrive/OUT-K3/VOCAB'
INPUT_DIR='/content/drive/MyDrive/EHR-GIT/DATASET'
# DATA_DIR = '/content/drive/MyDrive/EHR/DATA'
MIMIC_3_DIR = '/content/drive/MyDrive/EHR-GIT/MIMIC3'

In [ ]:
cd /content/drive/MyDrive/OUT-K3

/content/drive/MyDrive/OUT-K3


NOTEEVENTS file contains clinical notes of the patient along with other details like subject_id, hadm_id, category of the clinical notes etc.

In [4]:
df = pd.read_csv('%s/NOTEEVENTS.csv' % INPUT_DIR)

<ipython-input-4-30671e7fae59>:1: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('%s/NOTEEVENTS.csv' % INPUT_DIR)


In [5]:
df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118.0,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
3,177,13702,196489.0,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
4,178,26880,135453.0,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...


Preprocess the text - remove punctuation, convert to lowercase, remove less important words and remove unnecessary sections like medications

In [ ]:

from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')

with open('%s/NOTEEVENTS.csv' % INPUT_DIR, 'r') as infile:
  with open('%s/NOTES_FILTERED.csv' % MIMIC_3_DIR, 'w') as outfile:
    outfile.write(','.join(['SUBJECT_ID', 'HADM_ID', 'CHARTDATE', 'CHARTTIME', 'STORETIME', 'DESCRIPTION', 'CGID', 'ISERROR', 'TEXT']) + '\n')
    reader=csv.reader(infile)
    next(reader)  
    count=0  
    flag=0
    
    for row in reader:
      text_new=" "
      flag=flag+1     
      if row[6]=='Discharge summary':
        text = row[10]
        for line in text.split('\n\n'):
          tokens = [t.lower() for t in tokenizer.tokenize(line) if (t.isnumeric()==False and len(t)>3 and t.lower() not in['this','also','date','birth','admission','discharge','studies','name'] )]
          # print(tokens)
          # l1=["medications","admission"]
          if ("medications" in tokens and ("discharge" in tokens or "admission" in tokens)):
            continue
          else:
            # print(tokens)
            new_text=  ' '.join(tokens) 
            text_new+=new_text+' '
        # print(text_new)
        outfile.write(','.join([ row[1], row[2], row[3], row[4], row[5], row[7], row[8], row[9], text_new]) + '\n') 
        # if(flag==2):
        #   break
    

In [ ]:
dfl = pd.read_csv('%s/NOTES_FILTERED.csv' % MIMIC_3_DIR)
dfl.head()

Preprocess the data to extract the necessary information from the dataset.


In [ ]:
df = pd.read_csv('%s/NOTES_FILTERED.csv' % MIMIC_3_DIR)
hadm_ids = set(df['HADM_ID'])
with open('%s/DIAGNOSES_ICD.csv' % INPUT_DIR, 'r') as lf:
    with open('%s/CODES_FILTERED.csv' % MIMIC_3_DIR , 'w') as of:
        w = csv.writer(of)
        w.writerow(['SUBJECT_ID', 'HADM_ID', 'ICD9_CODE'])
        r = csv.reader(lf)
        #header
        next(r)
        for i,row in enumerate(r):
            hadm_id = int(row[2])
            #print(hadm_id)
            #break
            if hadm_id in hadm_ids:
                w.writerow(row[1:3] + [row[4]])

Sort the filtered NOTES and CODES

In [ ]:
# df = pd.read_csv('%s/NOTES_FILTERED.csv' % MIMIC_3_DIR)
df = df.sort_values(['SUBJECT_ID', 'HADM_ID'])
df.to_csv('%s/NOTES_FILTERED_SORTED.csv' % MIMIC_3_DIR, index=False)

In [ ]:
df2 = pd.read_csv('%s/CODES_FILTERED.csv' % MIMIC_3_DIR)
df2 = df2.sort_values(['SUBJECT_ID', 'HADM_ID'])
df2.to_csv('%s/CODES_FILTERED_SORTED.csv' % MIMIC_3_DIR, index=False)

In [ ]:
def next_notes2(notesfile):

    notes_reader = csv.reader(notesfile)

    arr = []

    for (i,v) in enumerate(notes_reader):

        if(i==0):
            continue

        if(i==1):

            sid = int(v[0])
            hid = int(v[1])
            text = v[8]
            continue
    
        cur_sid = int(v[0])
        cur_hid = int(v[1])
        cur_text = v[8]

        if(hid==cur_hid):
            text += " " + cur_text
        
        else:
            yield(sid,text,hid)
            hid = cur_hid
            sid = cur_sid
            text = cur_text
        
    yield(sid,text,hid)

def next_labels2(labelsfile):

    for (i,v) in enumerate(csv.reader(labelsfile)):

        if(i==0):
            continue

        if(i==1):

            sid = int(v[0])
            hid = int(v[1])
            code = [v[2]]
            continue
            
        cur_sid = int(v[0])
        cur_hid = int(v[1])
        cur_code = v[2]

        if(hid==cur_hid):
            code.append(cur_code)
            
        else:
            yield(sid,code,hid)
            hid = cur_hid
            sid = cur_sid
            code = [cur_code]

    yield(sid,code,hid)

def concat_data(labels, notes,out):
    
    with open(labels, 'r') as lf:
        print("CONCATENATING labels with notes")
        with open(notes, 'r') as nf:
            with open(out, 'w') as rf:
                w = csv.writer(rf)
                w.writerow(['SUBJECT_ID', 'HADM_ID', 'TEXT', 'LABELS'])

                labels_gen = next_labels2(lf)
                notes_gen = next_notes2(nf)

                for i, (subj_id, text, hadm_id) in enumerate(notes_gen):
                    if i % 10000 == 0:
                        print(str(i) + " done")
                    cur_subj, cur_labels, cur_hadm = next(labels_gen)

                    if cur_hadm == hadm_id:
                        w.writerow([subj_id, str(hadm_id), text, ' '.join(cur_labels)])
                    else:
                        print("Data is not sorted correctly. Cannot find matching hadm_id")
                        break


In [ ]:
concat_data('%s/CODES_FILTERED_SORTED.csv' %MIMIC_3_DIR , '%s/NOTES_FILTERED_SORTED.csv' %MIMIC_3_DIR,'%s/NOTES_CODES.csv' %MIMIC_3_DIR)

CONCATENATING labels with notes
0 done
10000 done
20000 done
30000 done
40000 done
50000 done


In [ ]:
df1 = pd.read_csv('%s/NOTES_CODES.csv' %MIMIC_3_DIR)
df1.head()

### TEST Train splitting

In [ ]:
def divide(file):
  with open(file, 'r') as f:
    reader = csv.reader(f)
    length = len(list(reader))
    train_length = (length*8)//10
    dev_length = (length-train_length)//2
    test_length = length - (train_length + dev_length)

    return length, train_length, dev_length, test_length


def split_hadm(filename):
  length, train_length, dev_length, test_length = divide(filename)
  with open(filename, 'r') as f:
    with open('%s/train_hadm_ids.csv' %MIMIC_3_DIR, 'w') as train_hadm_ids:
      with open('%s/dev_hadm_ids.csv' %MIMIC_3_DIR, 'w') as dev_hadm_ids:
        with open('%s/test_hadm_ids.csv' %MIMIC_3_DIR, 'w') as test_hadm_ids:
          train_hadm_ids.write('HADM_IDS' + "\n")
          dev_hadm_ids.write('HADM_IDS' + "\n")
          test_hadm_ids.write('HADMS' + "\n")
          re = csv.reader(f)
          next(re)
          i=0
          for row in re:            
            if(i<train_length):
              train_hadm_ids.write(row[1] + "\n")              
            
            if(i>=train_length and i<(train_length+dev_length)):
              dev_hadm_ids.write(row[1] + "\n")              

            if(i>=(train_length+dev_length) and i<length):
              test_hadm_ids.write(row[1]+ "\n")

            i+=1

def splitdata2(filename):
  split_hadm(filename)
  train = '%s/train_discharge_split.csv' %MIMIC_3_DIR
  dev = '%s/dev_discharge_split.csv' %MIMIC_3_DIR
  test = '%s/test_discharge_split.csv' %MIMIC_3_DIR
  file_train = open(train, 'w')
  file_dev = open(dev, 'w')
  file_test = open(test, 'w')
  file_train.write(','.join(['SUBJECT_ID', 'HADM_ID', 'TEXT', 'LABELS']) + "\n")
  file_dev.write(','.join(['SUBJECT_ID', 'HADM_ID', 'TEXT', 'LABELS']) + "\n")
  file_test.write(','.join(['SUBJECT_ID', 'HADM_ID', 'TEXT', 'LABELS']) + "\n")

  Hadm_Ids = {}

  for splt in ['train', 'dev', 'test']:
    Hadm_Ids[splt] = set()
    with open('%s/%s_hadm_ids.csv' %(MIMIC_3_DIR,splt), 'r') as f:
      for line in f:
        Hadm_Ids[splt].add(line.rstrip())

  with open(filename, 'r') as fn:
    reader = csv.reader(fn)
    next(reader)
    i=0
    current_hadmid = 0
    for row in reader:
      if i % 10000 == 0:
        print(str(i) + " read")
      HadmId = row[1]
      if HadmId in Hadm_Ids['train']:
        file_train.write(','.join(row) + "\n")
      elif HadmId in Hadm_Ids['dev']:
        file_dev.write(','.join(row) + "\n")
      elif HadmId in Hadm_Ids['test']:
        file_test.write(','.join(row) + "\n")

      i += 1

  file_train.close()
  file_dev.close()
  file_test.close()

  return train, dev, test

In [ ]:
tr, dv, te = splitdata2('%s/NOTES_CODES.csv' %MIMIC_3_DIR)
# print(tr)
# print(dv)
# print(te)

0 read
10000 read
20000 read
30000 read
40000 read
50000 read


In [ ]:
from collections import defaultdict
from scipy.sparse import csr_matrix

def build_vocab(vocab_min, infile, vocab_filename):
    """
        INPUTS:
            vocab_min: how many documents a word must appear in to be kept
            infile: (training) data file to build vocabulary from
            vocab_filename: name for the file to output
    """
    with open(infile, 'r') as csvfile:
        reader = csv.reader(csvfile)
        #header
        next(reader)

        #0. read in data
        print("reading in data...")
        #holds number of terms in each document
        note_numwords = []
        #indices where notes start
        note_inds = [0]
        #indices of discovered words
        indices = []
        #holds a bunch of ones
        data = []
        #keep track of discovered words
        vocab = {}
        #build lookup table for terms
        num2term = {}
        #preallocate array to hold number of notes each term appears in
        note_occur = np.zeros(400000, dtype=int)
        i = 0
        for row in reader:
            text = row[2]
            numwords = 0
            for term in text.split():
                #put term in vocab if it's not there. else, get the index
                index = vocab.setdefault(term, len(vocab))
                indices.append(index)
                num2term[index] = term
                data.append(1)
                numwords += 1
            #record where the next note starts
            note_inds.append(len(indices))
            indset = set(indices[note_inds[-2]:note_inds[-1]])
            #go thru all the word indices you just added, and add to the note occurrence count for each of them
            for ind in indset:
                note_occur[ind] += 1
            note_numwords.append(numwords)
            i += 1
        #clip trailing zeros
        note_occur = note_occur[note_occur>0]

        #turn vocab into a list so indexing doesn't get fd up when we drop rows
        vocab_list = np.array([word for word,ind in sorted(vocab.items(), key=operator.itemgetter(1))])

        #1. create sparse document matrix
        C = csr_matrix((data, indices, note_inds), dtype=int).transpose()
        #also need the numwords array to be a sparse matrix
        note_numwords = csr_matrix(1. / np.array(note_numwords))
        
        #2. remove rows with less than 3 total occurrences
        print("removing rare terms")
        #inds holds indices of rows corresponding to terms that occur in < 3 documents
        inds = np.nonzero(note_occur >= vocab_min)[0]
        print(str(len(inds)) + " terms qualify out of " + str(C.shape[0]) + " total")
        #drop those rows
        C = C[inds,:]
        note_occur = note_occur[inds]
        vocab_list = vocab_list[inds]

        print("writing output")
        with open(vocab_filename, 'w') as vocab_file:
            for word in vocab_list:
                vocab_file.write(word + "\n")

def build_vocab2(vocab_min, infile, vocab_filename):
    """
        INPUTS:
            vocab_min: how many documents a word must appear in to be kept
            infile: (training) data file to build vocabulary from
            vocab_filename: name for the file to output
    """
    with open(infile, 'r') as csvfile:
        reader = csv.reader(csvfile)
        #header
        next(reader)

        #0. read in data
        print("reading in data...")
        #holds number of terms in each document
        note_numwords = []
        #indices where notes start
        note_inds = [0]
        #indices of discovered words
        indices = []
        #holds a bunch of ones
        data = []
        #keep track of discovered words
        vocab = {}
        #build lookup table for terms
        num2term = {}
        #preallocate array to hold number of notes each term appears in
        note_occur = np.zeros(400000, dtype=int)
        i = 0
        for row in reader:
            text = row[2]
            numwords = 0
            for term in text.split():
                #put term in vocab if it's not there. else, get the index
                index = vocab.setdefault(term, len(vocab))
                indices.append(index)
                num2term[index] = term
                data.append(1)
                numwords += 1
            #record where the next note starts
            note_inds.append(len(indices))
            indset = set(indices[note_inds[-2]:note_inds[-1]])
            #go thru all the word indices you just added, and add to the note occurrence count for each of them
            for ind in indset:
                note_occur[ind] += 1
            note_numwords.append(numwords)
            i += 1
        #clip trailing zeros
        note_occur = note_occur[note_occur>0]

        #turn vocab into a list so indexing doesn't get fd up when we drop rows
        vocab_list = np.array([word for word,ind in sorted(vocab.items(), key=operator.itemgetter(1))])

        #1. create sparse document matrix
        C = csr_matrix((data, indices, note_inds), dtype=int).transpose()
        #also need the numwords array to be a sparse matrix
        note_numwords = csr_matrix(1. / np.array(note_numwords))
        
        #2. remove rows with less than 3 total occurrences
        print("removing rare terms")
        #inds holds indices of rows corresponding to terms that occur in < 3 documents
        inds = np.nonzero(note_occur >= vocab_min)[0]
        print(str(len(inds)) + " terms qualify out of " + str(C.shape[0]) + " total")
        #drop those rows
        C = C[inds,:]
        note_occur = note_occur[inds]
        vocab_list = vocab_list[inds]

        print("writing output")
        with open(vocab_filename, 'w') as vocab_file:
            for word in vocab_list:
                vocab_file.write(word + "\n")

In [ ]:
vocab_min = 3
vname = '%s/vocab.csv' %MIMIC_3_DIR
build_vocab2(vocab_min, tr, vname)

reading in data...
removing rare terms
43435 terms qualify out of 121309 total
writing output


In [ ]:
df=pd.read_csv('%s/vocab.csv' %MIMIC_3_DIR)
df.head(10)

In [ ]:
for splt in ['train', 'dev', 'test']:
    df = pd.read_csv('%s/%s_discharge_split.csv' % (MIMIC_3_DIR,splt))
    df['length'] = df.apply(lambda row: len(str(row['TEXT']).split()), axis=1)
    df = df.sort_values(['length'])
    df.to_csv('%s/%s_full.csv' % (MIMIC_3_DIR,splt), index=False)

In [ ]:
import gensim.models.word2vec as w2v

class ProcessedIter(object):

    def __init__(self, Y, filename):
        self.filename = filename

    def __iter__(self):
        with open(self.filename) as f:
            r = csv.reader(f)
            next(r)
            for row in r:
                yield (row[8].split())

def word_embeddings(Y, notes_file, embedding_size, min_count, n_iter):
    # modelname = 
    sentences = ProcessedIter(Y, notes_file)

    model = w2v.Word2Vec(vector_size=embedding_size, min_count=min_count, workers=4, epochs=n_iter)
    print("building word2vec vocab on %s..." % (notes_file))
    
    model.build_vocab(sentences)
    print("training...")
    model.train(sentences, total_examples=model.corpus_count, epochs=model.epochs)
    out_file = "%s/processed_%s.w2v" % (MIMIC_3_DIR,Y)
    print("writing embeddings to %s" % (out_file))
    model.save(out_file)
    return out_file

Building w2v file from NOTES filtered




In [ ]:
w2v_file = word_embeddings('full', '%s/NOTES_FILTERED.csv' %MIMIC_3_DIR, 100, 0, 5)

building word2vec vocab on /content/drive/MyDrive/OUT-K3/MIMIC3/NOTES_FILTERED.csv...
training...
writing embeddings to /content/drive/MyDrive/OUT-K3/MIMIC3/processed_full.w2v


In [ ]:
import gensim.models
model = gensim.models.Word2Vec.load('%s/processed_full.w2v' %MIMIC_3_DIR)
wv = model.wv
wv.similarity('fever','cold')

0.22310127

In [ ]:
import gensim.models
import os
from tqdm import tqdm

PAD_CHAR = "**PAD**"

def gensim_to_embeddings(wv_file, vocab_file, Y, outfile=None):
    model = gensim.models.Word2Vec.load(wv_file)
    wv = model.wv
    #free up memory
    del model

    vocab = set()
    with open(vocab_file, 'r') as vocabfile:
        for i,line in enumerate(vocabfile):
            line = line.strip()
            if line != '':
                vocab.add(line)
    ind2w = {i+1:w for i,w in enumerate(sorted(vocab))}

    W, words = build_matrix(ind2w, wv)

    if outfile is None:
        outfile = wv_file.replace('.w2v', '.embed')

    #smash that save button
    save_embeddings(W, words, outfile)

def build_matrix(ind2w, wv):
    """
        Go through vocab in order. Find vocab word in wv.index2word, then call wv.word_vec(wv.index2word[i]).
        Put results into one big matrix.
        Note: ind2w starts at 1 (saving 0 for the pad character), but gensim word vectors starts at 0
    """
    W = np.zeros((len(ind2w)+1, len(wv.word_vec(wv.index_to_key[0])) ))
    words = [PAD_CHAR]
    W[0][:] = np.zeros(len(wv.word_vec(wv.index_to_key[0])))
    for idx, word in tqdm(ind2w.items()):
        if idx >= W.shape[0]:
            break    
        W[idx][:] = wv.word_vec(word)
        words.append(word)
    return W, words

def save_embeddings(W, words, outfile):
    with open(outfile, 'w') as o:
        #pad token already included
        for i in range(len(words)):
            line = [words[i]]
            line.extend([str(d) for d in W[i]])
            o.write(" ".join(line) + "\n")

In [ ]:
gensim_to_embeddings('%s/processed_full.w2v' %MIMIC_3_DIR, '%s/vocab.csv' %MIMIC_3_DIR,'full')

<ipython-input-11-162f7d886443>:35: DeprecationWarning: Call to deprecated `word_vec` (Use get_vector instead).
  W = np.zeros((len(ind2w)+1, len(wv.word_vec(wv.index_to_key[0])) ))
<ipython-input-11-162f7d886443>:37: DeprecationWarning: Call to deprecated `word_vec` (Use get_vector instead).
  W[0][:] = np.zeros(len(wv.word_vec(wv.index_to_key[0])))
  0%|          | 0/43435 [00:00<?, ?it/s]<ipython-input-11-162f7d886443>:41: DeprecationWarning: Call to deprecated `word_vec` (Use get_vector instead).
  W[idx][:] = wv.word_vec(word)
100%|██████████| 43435/43435 [00:00<00:00, 205027.90it/s]


In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from tqdm import tqdm

def vocab_index_descriptions(vocab_file, vectors_file):
    #load lookups
    vocab = set()
    with open(vocab_file, 'r') as vocabfile:
        for i,line in enumerate(vocabfile):
            line = line.strip()
            if line != '':
                vocab.add(line)
    ind2w = {i+1:w for i,w in enumerate(sorted(vocab))}
    w2ind = {w:i for i,w in ind2w.items()}
    desc_dict = load_code_descriptions()
        
    tokenizer = RegexpTokenizer(r'\w+')

    with open(vectors_file, 'w') as of:
        w = csv.writer(of, delimiter=' ')
        w.writerow(["CODE", "VECTOR"])
        for code, desc in tqdm(desc_dict.items()):
            #same preprocessing steps as in get_discharge_summaries
            tokens = [t.lower() for t in tokenizer.tokenize(desc) if not t.isnumeric()]
            inds = [w2ind[t] if t in w2ind.keys() else len(w2ind)+1 for t in tokens]
            w.writerow([code] + [str(i) for i in inds])


def load_code_descriptions(version='mimic3'):
    #load description lookup from the appropriate data files
    desc_dict = defaultdict(str)
    with open("%s/D_ICD_DIAGNOSES.csv" %DATA_DIR, 'r') as descfile:
        r = csv.reader(descfile)
        #header
        next(r)
        for row in r:
            code = row[1]
            desc = row[-1]
            desc_dict[reformat(code, True)] = desc
    with open("%s/D_ICD_PROCEDURES.csv" %DATA_DIR, 'r') as descfile:
        r = csv.reader(descfile)
        #header
        next(r)
        for row in r:
            code = row[1]
            desc = row[-1]
            if code not in desc_dict.keys():
                desc_dict[reformat(code, False)] = desc
    with open('%s/ICD9_descriptions' %DATA_DIR, 'r') as labelfile:
        for i,row in enumerate(labelfile):
            row = row.rstrip().split()
            code = row[0]
            if code not in desc_dict.keys():
                desc_dict[code] = ' '.join(row[1:])
    return desc_dict

def reformat(code, is_diag):
    """
        Put a period in the right place because the MIMIC-3 data files exclude them.
        Generally, procedure codes have dots after the first two digits, 
        while diagnosis codes have dots after the first three digits.
    """
    code = ''.join(code.split('.'))
    if is_diag:
        if code.startswith('E'):
            if len(code) > 4:
                code = code[:4] + '.' + code[4:]
        else:
            if len(code) > 3:
                code = code[:3] + '.' + code[3:]
    else:
        code = code[:2] + '.' + code[2:]
    return code

In [ ]:
vocab_index_descriptions('%s/vocab.csv' % MIMIC_3_DIR,'%s/description_vectors.vocab' %MIMIC_3_DIR)

100%|██████████| 22267/22267 [00:00<00:00, 136954.03it/s]


In [ ]:
!ls

sample_data
